In [1]:
# import pandas module
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime as dt

In [2]:
from bokeh.io import output_notebook
from bokeh.models.widgets import Panel, Tabs
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.transform import dodge
from bokeh.models import Span, CheckboxGroup, CustomJS, CheckboxButtonGroup, Legend
from bokeh.palettes import Turbo256
output_notebook()

Loading BokehJS ...

In [32]:
participants = ['a','b','c','d','e']

def load_files(file_name):
       
    df_list = []

    for i in participants:
        df_list.append(pd.read_csv(file_name.format(i)))

    df = pd.concat(df_list)
    
    return df

mental = load_files('data/mental_files/mental_{}.csv')
mental_mean = load_files('data/mental_files/mental_{}_mean.csv')
mental['date'] = pd.to_datetime(mental['date'], format='%Y%m%d')

baseline = load_files('data/mental_files/baseline_{}.csv')
baseline_mean = load_files('data/mental_files/baseline_{}_mean.csv')
baseline['date'] = pd.to_datetime(baseline['date'], format='%Y%m%d')

vegan = load_files('data/mental_files/vegan_{}.csv')
vegan_mean = load_files('data/mental_files/vegan_{}_mean.csv')
vegan['date'] = pd.to_datetime(vegan['date'], format='%Y%m%d')

mental_df = {}
baseline_df = {}
vegan_df = {}

for i in participants:
    mental_df["mental_health_{0}".format(i)] = mental[mental['Subject'] == i]
    baseline_df["baseline_{0}".format(i)] = baseline[baseline['Subject'] == i]     
    vegan_df["vegan_{0}".format(i)] = vegan[vegan['Subject'] == i]

In [33]:
def line_plot(df):
    
    p = figure(plot_width=1000, plot_height=400, y_range=(0,7) , x_axis_type='datetime')
    
    colors = list(Turbo256)
    index = 0

    for (columnName, columnData) in df.iteritems():
        if index != 0:
            p.line(df['date'], df[columnName], line_width=2, color=colors[index], legend_label=columnName)
        index = index + 1
        
    # add the line to see the border between baseline and vegan
    border = time.mktime(dt(2020, 10, 26, 3, 0, 0).timetuple())*1000
    daylight_savings_start = Span(location=border, dimension='height', line_color='hotpink', line_width=3)
    p.add_layout(daylight_savings_start)

    p.add_layout(p.legend[0], 'right')
    p.legend.click_policy= "hide"

    return p

def tabs_plot():
    # initiate list for the tabs
    tab_list = []

    # loop through the dict of y values and descriptions
    for df in mental_df:
        # create plot for every loop
        p = line_plot(mental_df[df])
        # create tab for every loop
        tab = Panel(child=p, title='Line plot of {}'.format(df))
        # add tab to list
        tab_list.append(tab)

    tabs = Tabs(tabs=tab_list)
    show(tabs)

tabs_plot()